<a href="https://colab.research.google.com/github/muralisreedhar/data-analysis/blob/main/SCPCP000023_SCPCP000758.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scanpy

In [ ]:
!pip install gseapy

In [ ]:
!pip install infercnvpy

In [ ]:
import csv
import gzip
import os
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
# For GSEA, you might use gseapy
import gseapy
import infercnvpy as cnv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
adata = sc.read_h5ad('/content/drive/MyDrive/ALSF/SCPCP000023/SCPCS000758/SCPCL001120_processed_rna.h5ad')

In [ ]:
adata

# Step 1: Initial Review and Preliminary Assessment

In [ ]:
# Review existing clusters and annotations
# Visualize UMAP colored by existing annotations (e.g., consensus_celltype_annotation)
print("Step 1.1: Visualizing existing clusters and annotations")
sc.pl.umap(adata, color=['cluster', 'cellassign_celltype_annotation'], legend_loc='on data', title=['Clusters', 'Cellassign Celltype Annotation'])

In [ ]:
# Perform differential gene expression analysis for initial clusters
# This uses the 'cluster' column in adata.obs
print("Step 1.2: Performing differential gene expression for initial clusters")
sc.tl.rank_genes_groups(adata, 'cluster', method='wilcoxon')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False, title="Top 25 DEGs per Cluster")

In [ ]:
# You can access the DGE results as a pandas DataFrame:
dge_results = pd.DataFrame(adata.uns['rank_genes_groups']['names'])
print(dge_results.head())

In [ ]:
# Option 2: If adata.var_names are Ensembl IDs and gene symbols are in adata.var['gene_symbol']
osteosarcoma_marker_symbols = ['COL1A1', 'CXCL12', 'MEPE', 'COL2A1']
# Get the var_names (e.g. Ensembl IDs) for these symbols
# Create a mapping from symbol to var_name if needed, or directly use boolean indexing
# This assumes 'gene_symbol' is a column in adata.var
osteosarcoma_markers_present = adata.var_names[adata.var['gene_symbol'].isin(osteosarcoma_marker_symbols)].tolist()

In [ ]:
# Assess expression of general osteosarcoma markers
if osteosarcoma_markers_present:
    print("Step 1.3: Assessing expression of general osteosarcoma markers")
    sc.pl.umap(adata, color=osteosarcoma_markers_present, title=[f"{marker} Expression" for marker in osteosarcoma_marker_symbols if adata.var_names[adata.var['gene_symbol'] == marker][0] in osteosarcoma_markers_present])
    sc.pl.dotplot(adata, osteosarcoma_markers_present, groupby='cluster', title="Osteosarcoma Markers Expression by Cluster")
else:
    print("Step 1.3: None of the specified osteosarcoma markers were found in the data using adata.var['gene_symbol'].")

In [ ]:
# Assess expression of osteoblastic or chondroblastic subtype markers
osteoblastic_markers_symbols = ['RUNX2', 'COL3A1']
chondroblastic_markers_symbols = ['ACAN', 'COL2A1', 'SOX9']

osteoblastic_markers_present = adata.var_names[adata.var['gene_symbol'].isin(osteoblastic_markers_symbols)].tolist()
chondroblastic_markers_present = adata.var_names[adata.var['gene_symbol'].isin(chondroblastic_markers_symbols)].tolist()

if osteoblastic_markers_present:
    print("Assessing expression of osteoblastic markers")
    sc.pl.umap(adata, color=osteoblastic_markers_present, title=[f"{marker} Expression" for marker in osteoblastic_markers_symbols if adata.var_names[adata.var['gene_symbol'] == marker][0] in osteoblastic_markers_present])
    sc.pl.dotplot(adata, osteoblastic_markers_present, groupby='cluster', title="Osteoblastic Markers Expression by Cluster")
if chondroblastic_markers_present:
    print("Assessing expression of chondroblastic markers")
    sc.pl.umap(adata, color=chondroblastic_markers_present, title=[f"{marker} Expression" for marker in chondroblastic_markers_symbols if adata.var_names[adata.var['gene_symbol'] == marker][0] in chondroblastic_markers_present])
    sc.pl.dotplot(adata, chondroblastic_markers_present, groupby='cluster', title="Chondroblastic Markers Expression by Cluster")

# Step 2: Prepare Gene Order File for InferCNV

In [ ]:
external_gene_order_df = pd.read_csv('/content/drive/MyDrive/ALSF/gencode_v19_gene_pos.txt', sep='\t', header=None, names=['gene_symbol_external', 'chromosome', 'start', 'end'])

In [ ]:
external_gene_order_df.head()

In [ ]:
gene_map_df = adata.var[['gene_symbol']].copy()

In [ ]:
gene_map_df = gene_map_df.reset_index()
gene_map_df.rename(columns={gene_map_df.columns[0]: 'Ensembl_ID'}, inplace=True)
gene_map_df.head()

In [ ]:
# Clean the map: drop rows with missing gene symbols, drop duplicate symbols (keeping first Ensembl ID)
gene_map_df.dropna(subset=['gene_symbol'], inplace=True)
gene_map_df.drop_duplicates(subset=['gene_symbol'], keep='first', inplace=True) # Important for a clean merge

In [ ]:
gene_map_df.shape

In [ ]:
# Merge based on gene symbols from the external file and our map
merged_gene_order_df = pd.merge(
    external_gene_order_df,
    gene_map_df,
    left_on='gene_symbol_external',
    right_on='gene_symbol',
    how='inner' # Keep only genes present in both
)
print(f"Merged gene order data. Shape: {merged_gene_order_df.shape}")
print(merged_gene_order_df.head())

In [ ]:
final_gene_order_for_infercnpy_df = merged_gene_order_df[['Ensembl_ID', 'chromosome', 'start', 'end']].copy()
# Ensure Ensembl_IDs are unique (should be if gene_map_df was correctly prepared and Ensembl_IDs in adata are unique)
final_gene_order_for_infercnpy_df.drop_duplicates(subset=['Ensembl_ID'], keep='first', inplace=True)

In [ ]:
# Sort by chromosome and start position (critical for infercnpy)
# Handle chromosome strings like 'chrX', 'chrY', 'chrM' by converting 'chr' prefix to numerical for sorting
final_gene_order_for_infercnpy_df['chromosome_numerical'] = final_gene_order_for_infercnpy_df['chromosome'].astype(str).str.replace('chr', '')
# Replace X, Y, M with high numbers for sorting after numerical chromosomes
replace_map = {'X': 100, 'Y': 101, 'M': 102, 'MT': 102} # MT for mitochondrial
for k, v in replace_map.items():
    final_gene_order_for_infercnpy_df['chromosome_numerical'] = final_gene_order_for_infercnpy_df['chromosome_numerical'].replace(k, str(v))

In [ ]:
# Attempt to convert to numeric, coercing errors for any non-standard chromosome names
final_gene_order_for_infercnpy_df['chromosome_numerical'] = pd.to_numeric(final_gene_order_for_infercnpy_df['chromosome_numerical'], errors='coerce')
final_gene_order_for_infercnpy_df.dropna(subset=['chromosome_numerical'], inplace=True) # Drop if not convertible

final_gene_order_for_infercnpy_df = final_gene_order_for_infercnpy_df.sort_values(
    by=['chromosome_numerical', 'start']
).drop(columns=['chromosome_numerical'])

In [ ]:
# Save to a temporary file (tab-separated, no header, specific columns)
processed_gene_order_file_path = "/content/drive/MyDrive/ALSF/processed_gene_order_for_infercnpy.txt"
final_gene_order_for_infercnpy_df[['Ensembl_ID', 'chromosome', 'start', 'end']].to_csv(
    processed_gene_order_file_path, sep='\t', index=False, header=False
)
print(f"Processed gene order file saved to: {processed_gene_order_file_path}. Shape: {final_gene_order_for_infercnpy_df.shape}")
print(final_gene_order_for_infercnpy_df.head())

# Step 3: Define Reference Cell Types

In [ ]:
reference_cell_types = adata.obs['cellassign_celltype_annotation'].unique()

In [ ]:
adata.obs['cnv_ref_group'] = adata.obs['cellassign_celltype_annotation'].astype(str)
available_ref_types = [ct for ct in reference_cell_types if ct in adata.obs['cnv_ref_group'].unique()]
print(f"Using the following cell types as reference: {available_ref_types}")

# Step 4: Run Infercnpy

In [ ]:
# Subset AnnData to genes that are present in our final_gene_order_for_infercnpy_df
# These are the genes for which we have genomic coordinates.
common_genes_with_coords = final_gene_order_for_infercnpy_df['Ensembl_ID'].unique().tolist()
common_genes_in_adata = [gene for gene in adata.var_names if gene in common_genes_with_coords]

In [ ]:
adata_subset_for_cnv = adata[:, common_genes_in_adata].copy()
print(f"adata_subset_for_cnv created with {adata_subset_for_cnv.n_vars} genes and {adata_subset_for_cnv.n_obs} cells.")

In [ ]:
adata_subset_for_cnv.var_names

In [ ]:
# Add chromosome, start, and end information to adata_subset_for_cnv.var
# The gene order DataFrame needs to be indexed by 'Ensembl_ID' to align with adata_subset_for_cnv.var_names
gene_order_info_to_add = final_gene_order_for_infercnpy_df.set_index('Ensembl_ID')

In [ ]:
# Keep only the coordinate columns needed, ensure they are present
coordinate_cols = ['chromosome', 'start', 'end']
for col in coordinate_cols:
    if col not in gene_order_info_to_add.columns:
        raise ValueError(f"Column '{col}' is missing from the gene order information DataFrame.")

In [ ]:
# Join the coordinate information. This aligns based on the index (Ensembl_ID).
adata_subset_for_cnv.var = adata_subset_for_cnv.var.join(gene_order_info_to_add[coordinate_cols])

In [ ]:
# Verify that the columns were added and populated
print("Verifying .var columns after adding coordinates:")
print(adata_subset_for_cnv.var.head())
if adata_subset_for_cnv.var['chromosome'].isnull().any() or \
   adata_subset_for_cnv.var['start'].isnull().any() or \
   adata_subset_for_cnv.var['end'].isnull().any():
    print("Warning: Some genes in adata_subset_for_cnv.var have missing chromosome/start/end information after join.")
    print(adata_subset_for_cnv.var[adata_subset_for_cnv.var['chromosome'].isnull()].head())
    # Consider dropping these genes or investigating the merge if this happens unexpectedly
    # adata_subset_for_cnv = adata_subset_for_cnv[:, adata_subset_for_cnv.var['chromosome'].notnull()].copy()
    # print(f"Subsetted further to {adata_subset_for_cnv.n_vars} genes with complete coordinate info.")


In [ ]:
# Ensure the cnv_ref_group column is also in the subset if not already copied over by adata[:, common_genes_in_adata].copy()
# .copy() on a view should bring .obs along.
if 'cnv_ref_group' not in adata_subset_for_cnv.obs.columns and 'cnv_ref_group' in adata.obs.columns:
    adata_subset_for_cnv.obs['cnv_ref_group'] = adata.obs.loc[adata_subset_for_cnv.obs_names, 'cnv_ref_group']

available_ref_types = [ct for ct in reference_cell_types if ct in adata_subset_for_cnv.obs['cnv_ref_group'].unique()]
print(f"Using the following cell types as reference: {available_ref_types}")

In [ ]:
cnv.tl.infercnv(
        adata_subset_for_cnv,
        reference_key='cnv_ref_group',
        reference_cat=available_ref_types,
        # reference=None, # Default
        lfc_clip=3, # Default, can be adjusted
        window_size=100, # Default, can be adjusted
        step=10, # Default, can be adjusted
        dynamic_threshold=1.5, # Default, can be adjusted
        exclude_chromosomes=('chrX', 'chrY', 'chrM', 'chrMT'), # Default, adjust if needed e.g. ('chrX', 'chrY', 'chrM', 'chrMT')
        # chunksize=5000, # Default
        # n_jobs=None, # Default (all cores)
        inplace=True, # Default
        # layer=None, # Default (uses .X)
        key_added='cnv', # Default, stores in .obsm[f'X_{key_added}'] and .uns[key_added]
        # calculate_gene_values=False # Default
)

In [ ]:
# Transfer results back to the main adata object
cnv_key_obsm = f"X_{'cnv'}" # Default key_added is 'cnv'
cnv_key_uns = 'cnv'

In [ ]:
if cnv_key_obsm in adata_subset_for_cnv.obsm:
  cnv_data = adata_subset_for_cnv.obsm[cnv_key_obsm]
  print(f"Shape of infercnv output in .obsm['{cnv_key_obsm}']: {cnv_data.shape}")

In [ ]:
# The expected shape should be (number of cells, number of genes in adata_subset_for_cnv)
expected_shape = (adata_subset_for_cnv.n_obs, adata_subset_for_cnv.n_vars)
print(f"Expected shape based on adata_subset_for_cnv: {expected_shape}")

In [ ]:
if cnv_data.shape == expected_shape:
    print("Infercnv output shape matches expected shape. Proceeding with DataFrame creation.")
else:
    print("Warning: Infercnv output shape does not match expected shape.")

In [ ]:
cnv_output_matrix = adata_subset_for_cnv.obsm[cnv_key_obsm]
print(f"Shape of infercnv output matrix (adata_subset_for_cnv.obsm['{cnv_key_obsm}']): {cnv_output_matrix.shape}")
print(f"Number of genes in adata_subset_for_cnv (used if shapes matched): {adata_subset_for_cnv.n_vars}")

n_output_features = cnv_output_matrix.shape[1]

In [ ]:
print(f"inferCNVpy output has {n_output_features} features, matching the number of genes in the subset analysis.")
print("Assuming these features correspond to per-gene CNV values for the analyzed subset.")


In [ ]:
# Convert to dense if sparse
if hasattr(cnv_output_matrix, 'toarray'):
    cnv_output_dense = cnv_output_matrix.toarray()
else:
   cnv_output_dense = cnv_output_matrix

In [ ]:
window_names = [f'cnv_window_{i+1}' for i in range(n_output_features)]
cnv_window_matrix_df = pd.DataFrame(cnv_output_dense,
                                    index=adata_subset_for_cnv.obs_names,
                                    columns=window_names)

In [ ]:
adata.obsm['X_cnv_windows'] = pd.DataFrame(0.0,
                                           index=adata.obs_names,
                                           columns=window_names)

In [ ]:
adata.obsm['X_cnv_windows'].update(cnv_window_matrix_df)
adata.obsm['X_cnv_windows'] = adata.obsm['X_cnv_windows'].values
print(f"Stored CNV window values in adata.obsm['X_cnv_windows']")

In [ ]:
if cnv_key_uns in adata_subset_for_cnv.uns:
            adata.uns[cnv_key_uns] = adata_subset_for_cnv.uns[cnv_key_uns]
            print(f"CNV metadata transferred to adata.uns['{cnv_key_uns}']")
else:
            print(f"Warning: '{cnv_key_uns}' key not found in adata_subset_for_cnv.uns. Plotting might be affected.")

# Step 5: Visualize the results

In [ ]:
plot_adata = adata_subset_for_cnv
cnv_storage_key = 'cnv' # Default key_added

In [ ]:
# Check if the necessary data is present
obsm_key_to_check = f"X_{cnv_storage_key}"
if obsm_key_to_check not in plot_adata.obsm or cnv_storage_key not in plot_adata.uns:
    print(f"Skipping CNV plotting: Necessary data not found in plot_adata.")
    print(f"  Expected obsm key: '{obsm_key_to_check}' (Present: {obsm_key_to_check in plot_adata.obsm})")
    print(f"  Expected uns key: '{cnv_storage_key}' (Present: {cnv_storage_key in plot_adata.uns})")
else:
    print(f"Plotting CNV heatmap using .obsm['{obsm_key_to_check}'] and .uns['{cnv_storage_key}'].")

In [ ]:
print(f"Performing Leiden clustering on CNV profiles stored in .obsm['{obsm_key_to_check}']...")
sc.pp.neighbors(plot_adata, use_rep=obsm_key_to_check, key_added='cnv_neighbors') # Use the CNV data for neighbors
sc.tl.leiden(plot_adata, neighbors_key='cnv_neighbors', key_added='cnv_leiden_custom')
grouping_key = 'cnv_leiden_custom' # Use this new clustering for groupby
print(f"Using newly computed '{grouping_key}' for heatmap grouping.")

In [ ]:
cnv.pl.chromosome_heatmap(
                plot_adata,
                groupby=grouping_key,  # e.g., 'cnv_ref_group' or your preferred cell grouping
                use_rep=cnv_storage_key,  # This should match 'key_added' from tl.infercnv
                cmap='bwr',             # Common colormap for CNVs (blue-white-red)
                figsize=(18, 10),       # Adjust as needed
                # show=True,            # Default behavior is usually to show
                # save="cnv_chromosome_heatmap.pdf" # Uncomment to save
            )

# Step 6: Validate and Refine Tumor/Normal Assignments

In [ ]:
cnv_matrix_key = 'X_cnv_windows'
cnv_data_for_scoring = adata.obsm[cnv_matrix_key]

In [ ]:
# Calculate CNV activity score: sum of squared CNV values
adata.obs['cnv_activity_score'] = np.sum(np.square(cnv_data_for_scoring), axis=1)

In [ ]:
# Visualize the distribution
plt.figure(figsize=(8, 6))
sns.histplot(data=adata.obs, x='cnv_activity_score', hue='cnv_ref_group' if 'cnv_ref_group' in adata.obs else None, kde=True, multiple="stack")
plt.title('Distribution of CNV Activity Scores')
plt.xlabel('CNV Activity Score (sum of squares)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
reference_scores = adata.obs.loc[adata.obs['cnv_ref_group'].isin(reference_cell_types), 'cnv_activity_score']

In [ ]:
threshold = reference_scores.mean() + 2 * reference_scores.std()

In [ ]:
if 'cnv_ref_group' in adata.obs and reference_cell_types:
    is_reference_cell = adata.obs['cnv_ref_group'].isin(reference_cell_types)
    adata.obs.loc[is_reference_cell & (adata.obs['cnv_activity_score'] <= threshold), 'cnv_status'] = 'normal_cnv_ref'
    adata.obs.loc[is_reference_cell & (adata.obs['cnv_activity_score'] > threshold), 'cnv_status'] = 'tumor_cnv_ref_anomalous' # Suspicious reference

In [ ]:
adata.obs['cnv_status'] = adata.obs['cnv_status'].astype('category')
print("Distribution of 'cnv_status':")
print(adata.obs['cnv_status'].value_counts())

In [ ]:
osteosarcoma_marker_symbols = ['COL1A1', 'CXCL12', 'MEPE', 'COL2A1']
osteosarcoma_markers_present_in_adata = adata.var_names[adata.var['gene_symbol'].isin(osteosarcoma_marker_symbols)].tolist()

In [ ]:
osteosarcoma_markers_present_in_adata

In [ ]:
# To use gene symbols in plots when var_names are Ensembl IDs, create a temporary mapping for var_names_idx
    # This ensures dotplot/violin plot shows symbols if adata.var_names are not symbols.
    # However, sc.pl.dotplot/violin directly take a list of gene symbols if you pass them to the `var_names` argument
    # and your adata.var['gene_symbol'] is correctly set up.
    # The `groupby` parameter uses adata.obs.
    # sc.pl.dotplot(adata, osteosarcoma_markers_present_in_adata, groupby='cnv_status', title="Osteosarcoma Markers by CNV Status")
    # Better: use symbols directly in var_names for plotting if they exist in adata.var['gene_symbol']


In [ ]:
# sc.pl.dotplot(adata, var_names=osteosarcoma_marker_symbols, groupby='cnv_status', title="Osteosarcoma Markers by CNV Status", use_raw=False)
# sc.pl.violin(adata, var_names=osteosarcoma_marker_symbols, groupby='cnv_status', rotation=90, use_raw=False)

In [ ]:
adata

In [ ]:
adata.obs.cnv_status

In [ ]:
if len(adata.obs['cnv_status'].cat.categories) >= 2 and \
   ('tumor_cnv' in adata.obs['cnv_status'].value_counts() and adata.obs['cnv_status'].value_counts()['tumor_cnv'] > 1) and \
   ('normal_cnv' in adata.obs['cnv_status'].value_counts() and adata.obs['cnv_status'].value_counts()['normal_cnv'] > 1) : # Need at least 2 cells in each group for robust DGE
    print("Step 3.3: Performing DGE between CNV-defined tumor and normal cells")

In [ ]:
sc.tl.rank_genes_groups(adata, groupby='cnv_status', groups=['tumor_cnv_ref_anomalous'], reference='normal_cnv_ref', method='wilcoxon', key_added='rank_genes_tumor_vs_normal_cnv', use_raw=False)
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False, key='rank_genes_tumor_vs_normal_cnv', title="Top 25 DEGs (Tumor CNV vs Normal CNV)")

In [ ]:
sc.pl.umap(adata, color=['cnv_status', 'cnv_activity_score', 'cnv_ref_group'], title=['UMAP by CNV Status', 'UMAP by CNV Activity Score', 'UMAP by CNV Ref Group'], cmap='viridis')

In [ ]:
print("Step 4.1: Subsetting normal cells")
adata_normal = adata[adata.obs['cnv_status'] == 'normal_cnv_ref'].copy()

In [ ]:
# Re-cluster normal cells [cite: 34]
print("Step 4.2: Re-clustering normal cells")
sc.tl.pca(adata_normal, svd_solver='arpack')
sc.pp.neighbors(adata_normal, n_neighbors=10, n_pcs=min(30, adata_normal.n_obs-1)) # Adjust n_pcs if few cells
sc.tl.umap(adata_normal)
sc.tl.leiden(adata_normal, resolution=0.5, key_added='leiden_normal') # Adjust resolution as needed

In [ ]:
sc.pl.umap(adata_normal, color=['leiden_normal', 'cellassign_celltype_annotation'], title=['Leiden Clusters (Normal Subset)', 'Original Annotation (Normal Subset)'])